# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [13]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/michaelchuprin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/michaelchuprin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [15]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [22]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [23]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [24]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [25]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [26]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 64, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [27]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '6ef32c'. Skipping!
Property 'summary' already exists in node '3c9e7a'. Skipping!
Property 'summary' already exists in node 'd45cb6'. Skipping!
Property 'summary' already exists in node '6957cf'. Skipping!
Property 'summary' already exists in node 'cb9f61'. Skipping!
Property 'summary' already exists in node '644acc'. Skipping!
Property 'summary' already exists in node '33515c'. Skipping!
Property 'summary' already exists in node 'c7e0bd'. Skipping!
Property 'summary' already exists in node 'cc810b'. Skipping!
Property 'summary' already exists in node 'dc8444'. Skipping!
Property 'summary' already exists in node '7fb1fc'. Skipping!
Property 'summary' already exists in node '3c01c9'. Skipping!
Property 'summary' already exists in node '67c125'. Skipping!
Property 'summary' already exists in node '43c88d'. Skipping!
Property 'summary' already exists in node '4025a4'. Skipping!
Property 'summary' already exists in node 'ff78c6'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/48 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '6957cf'. Skipping!
Property 'summary_embedding' already exists in node '3c9e7a'. Skipping!
Property 'summary_embedding' already exists in node '6ef32c'. Skipping!
Property 'summary_embedding' already exists in node 'cb9f61'. Skipping!
Property 'summary_embedding' already exists in node '33515c'. Skipping!
Property 'summary_embedding' already exists in node 'd45cb6'. Skipping!
Property 'summary_embedding' already exists in node 'c7e0bd'. Skipping!
Property 'summary_embedding' already exists in node '644acc'. Skipping!
Property 'summary_embedding' already exists in node 'cc810b'. Skipping!
Property 'summary_embedding' already exists in node '7fb1fc'. Skipping!
Property 'summary_embedding' already exists in node 'dc8444'. Skipping!
Property 'summary_embedding' already exists in node '3c01c9'. Skipping!
Property 'summary_embedding' already exists in node '67c125'. Skipping!
Property 'summary_embedding' already exists in node '43c88d'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 86, relationships: 712)

We can save and load our knowledge graphs as follows.

In [28]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 712)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [29]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [30]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


##### ✅ Answers:

SingleHopSpecificQuerySynthesizer - Explicit information that needs only a single document (single node). 

MultiHopAbstractQuerySynthesizer - Implicit information that needs multuple documents (multiple nodes)

MultiHopSpecificQuerySynthesizer - Explicit information that needs multiple documents (multuple nodes)


Finally, we can use our `TestSetGenerator` to generate our testset!

In [31]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,"What does Korinek and Suh, 2024, discuss regar...",[Introduction ChatGPT launched in November 202...,"The paper by Korinek and Suh, 2024, studies th...",single_hop_specifc_query_synthesizer
1,Handa et al. (2025) ke baare mein kya bata sak...,[Table 1: ChatGPT daily message counts (millio...,Handa et al. (2025) ek research paper hai jo C...,single_hop_specifc_query_synthesizer
2,What is management and business?,[Variation by Occupation Figure 23 presents va...,Management and business occupations are broad ...,single_hop_specifc_query_synthesizer
3,How do Relationships influence the way users i...,[Conclusion This paper studies the rapid growt...,The provided context does not contain specific...,single_hop_specifc_query_synthesizer
4,H0w do work-related message sharing and usage ...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,The context indicates that users in highly pai...,multi_hop_abstract_query_synthesizer
5,"How do the societal effects of AI, such as the...",[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The rapid diffusion of ChatGPT, with 18 billio...",multi_hop_abstract_query_synthesizer
6,how occupation impact chatgpt use and activity...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,The context shows that users in highly paid pr...,multi_hop_abstract_query_synthesizer
7,How does the growth of ChatGPT usage in the US...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,"The context indicates that in the US, ChatGPT'...",multi_hop_specific_query_synthesizer
8,How does the fact that 700 million users engag...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The context indicates that by July 2025, ChatG...",multi_hop_specific_query_synthesizer
9,"How do Handa et al., 2025, describe the growth...",[<1-hop>\n\nIntroduction ChatGPT launched in N...,"Handa et al., 2025, report that since ChatGPT'...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [32]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '651fdb'. Skipping!
Property 'summary' already exists in node '465415'. Skipping!
Property 'summary' already exists in node '9f219e'. Skipping!
Property 'summary' already exists in node 'e736a8'. Skipping!
Property 'summary' already exists in node '312e0f'. Skipping!
Property 'summary' already exists in node '14eddd'. Skipping!
Property 'summary' already exists in node '85b227'. Skipping!
Property 'summary' already exists in node '040c9b'. Skipping!
Property 'summary' already exists in node '0433ad'. Skipping!
Property 'summary' already exists in node '9ead79'. Skipping!
Property 'summary' already exists in node 'e81119'. Skipping!
Property 'summary' already exists in node '95f460'. Skipping!
Property 'summary' already exists in node '3f0a23'. Skipping!
Property 'summary' already exists in node '677aa4'. Skipping!
Property 'summary' already exists in node '923e5e'. Skipping!
Property 'summary' already exists in node 'f2eac0'. Skipping!
Property

...

Property 'summary_embedding' already exists in node '85b227'. Skipping!
Property 'summary_embedding' already exists in node '312e0f'. Skipping!
Property 'summary_embedding' already exists in node '465415'. Skipping!
Property 'summary_embedding' already exists in node '9ead79'. Skipping!
Property 'summary_embedding' already exists in node '9f219e'. Skipping!
Property 'summary_embedding' already exists in node '651fdb'. Skipping!
Property 'summary_embedding' already exists in node '0433ad'. Skipping!
Property 'summary_embedding' already exists in node '040c9b'. Skipping!
Property 'summary_embedding' already exists in node '14eddd'. Skipping!
Property 'summary_embedding' already exists in node 'e81119'. Skipping!
Property 'summary_embedding' already exists in node 'e736a8'. Skipping!
Property 'summary_embedding' already exists in node '95f460'. Skipping!
Property 'summary_embedding' already exists in node '3f0a23'. Skipping!
Property 'summary_embedding' already exists in node '923e5e'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [41]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,How many messages were sent with 18 billion me...,[Introduction ChatGPT launched in November 202...,"By July 2025, 18 billion messages were being s...",single_hop_specifc_query_synthesizer
1,How is ChatGPT used for seeking info?,[Table 1: ChatGPT daily message counts (millio...,ChatGPT is used for seeking information about ...,single_hop_specifc_query_synthesizer
2,What is enginering and sciense in the context ...,[Variation by Occupation Figure 23 presents va...,Variation by Occupation Figure 23 presents var...,single_hop_specifc_query_synthesizer
3,Whaat is the role of Computer Programming in t...,[Conclusion This paper studies the rapid growt...,"According to the provided context, Computer Pr...",single_hop_specifc_query_synthesizer
4,How does the impact of occupation influence th...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,The context indicates that ChatGPT usage varie...,multi_hop_abstract_query_synthesizer
5,How do work-related message sharing patterns d...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,Variation by occupation shows that users in hi...,multi_hop_abstract_query_synthesizer
6,Based on message volume comparison and message...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,The data shows that non-work messages have gro...,multi_hop_abstract_query_synthesizer
7,How do occupation categories such as computer-...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,Variation in ChatGPT usage by occupation shows...,multi_hop_abstract_query_synthesizer
8,US ChatGPT use mostly for work or non-work and...,[<1-hop>\n\nConclusion This paper studies the ...,"The context shows that by July 2025, about 70%...",multi_hop_specific_query_synthesizer
9,"Based on the data, how has ChatGPT's usage in ...",[<1-hop>\n\nConclusion This paper studies the ...,"Between 2024 and 2025, ChatGPT's usage in the ...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [43]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data 3 - AIE8"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [44]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [45]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [47]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [48]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [49]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [50]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [51]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [52]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [53]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

'Based on the context provided, people are using AI, particularly generative AI like ChatGPT, in a variety of ways including performing workplace tasks either by augmenting or automating human labor. Generative AI is flexible and used both at work and outside work. Users seek to produce writing, software code, spreadsheets, and other digital products, distinguishing generative AI from traditional web search engines. Additionally, AI serves either as co-workers producing output or as co-pilots giving advice to improve human problem-solving productivity. There is also use in self-expression areas such as relationships, personal reflection, games, and role play, though these represent a smaller portion of usage. Overall, people use AI to ask questions, perform tasks, and express themselves, spanning professional, creative, and personal domains.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [54]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [55]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            ),
            # "clarity": (
            #     "Is this submission clear to the user taking into account the correct reference answer?"
            # )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dopeness_evaluator`:

##### ✅ Answers:

- `qa_evaluator`: semantic comparison between the answer and the reference answer

- `labeled_helpfulness_evaluator`: how helpful the answer is, i guess it checks the tone

- `dopeness_evaluator`: how dope the answer is, i guess how gen z it is


## LangSmith Evaluation

In [56]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'perfect-neck-85' at:
https://smith.langchain.com/o/d818ede9-bb51-42f5-821f-b15992a8e545/datasets/614eddd6-bbaf-45c6-9e36-1db976895173/compare?selectedSessions=2641aa1e-54ea-4513-872e-57eef6033af1




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"Based on the rapid growth of ChatGPT, which re...","Based on the provided context, by July 2025 Ch...",None,"By July 2025, ChatGPT's usage had expanded sig...",1,1,0,4.824434,67e0c56d-6bb3-4b88-a4d6-e03c37afeca8,aa65830d-0288-4b75-9f5c-96a62f6d48a0
1,"How do the 700 million users of ChatGPT, as of...","As of July 2025, ChatGPT had more than 700 mil...",None,"By July 2025, ChatGPT had been used weekly by ...",1,1,0,3.645114,f6aa2ef9-6ab9-4381-aa1f-1bd541b5e1b8,ddb43ddd-5271-451a-bbcc-72f64febc2d0
2,"Based on the data, how has ChatGPT's usage in ...","Based on the provided context, between 2024 an...",None,"Between 2024 and 2025, ChatGPT's usage in the ...",1,0,0,3.556988,f37baf0b-031a-4191-9efa-21182fd61d03,807b331d-82ca-4cda-9608-5dcf063d86f1
3,US ChatGPT use mostly for work or non-work and...,"Based on the context, US ChatGPT use is increa...",None,"The context shows that by July 2025, about 70%...",1,1,0,4.126528,dabdd65f-00c1-4d6d-b1d4-375117c30290,dec0a642-6c44-4594-8c09-7b62857e6b3a
4,How do occupation categories such as computer-...,Based on the provided context:\n\nOccupation c...,None,Variation in ChatGPT usage by occupation shows...,1,1,0,6.677597,c364651e-3bfd-48b2-9c74-16c42763fb79,77827be9-fafe-4566-a70b-68b199354291
5,Based on message volume comparison and message...,"Based on the provided context, the growth in n...",None,The data shows that non-work messages have gro...,1,1,0,3.087339,f3bacac5-c4de-4e9a-8fe1-81ccd9803485,b1c693b9-2b80-4ccd-abe2-8c39d33beb5e
6,How do work-related message sharing patterns d...,Based on the context provided:\n\nWork-related...,None,Variation by occupation shows that users in hi...,1,1,0,3.852668,24ebe91b-be63-4845-8230-4bd53c637ad2,909720c9-1728-45c7-934c-bc9aa7de5dcd
7,How does the impact of occupation influence th...,The impact of occupation significantly influen...,None,The context indicates that ChatGPT usage varie...,1,1,0,4.053071,b8cb9ed1-c9fb-4fba-b2dd-839d298f8281,8e5d9a9d-60e5-466d-8e61-1518b5c88e8b
8,Whaat is the role of Computer Programming in t...,"Based on the provided context, Computer Progra...",None,"According to the provided context, Computer Pr...",1,1,0,2.481464,5a40f1f4-2a26-4268-866e-650569a2286c,bf5bf6fa-5c15-4110-aef5-dba84436d452
9,What is enginering and sciense in the context ...,I don't know.,None,Variation by Occupation Figure 23 presents var...,0,0,0,1.081380,7583638f-68f7-40d9-8074-e34f7958a9e5,dc62b39a-0b17-446f-9ff5-7ec6a35c06ac


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [57]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [40]:
rag_documents = docs

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answers:

Several reasons
- it could be that larger chunks (more information in a single node) would allow simpler models to find the information they need from a single node rather than doing multiple hops

- it could be that smaller chunks would create better embeddings that are eaiser to search in the knowledge graph, allowing the embedding model to serve more relevant information

In [58]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answers:

- Like in the previous answer, it could serve more relevant information to queries

In [59]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [60]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [61]:
dopeness_rag_chain = {"context": itemgetter("question") | retriever, "question": itemgetter("question")} | dopeness_rag_prompt | llm | StrOutputParser()

Let's test it on the same output that we saw before.

In [62]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

'Alright, strap in for this AI-money glow-up: People are tapping into AI like ChatGPT to *level up their economic game* primarily through **decision support** in knowledge-heavy gigs. The docs flex that ChatGPT isn’t just spitting facts — it’s driving serious economic value by boosting how pros *analyze, strategize, and execute* complex work faster and smarter. \n\nThis isn’t just basic automation; it’s AI flexing its versatility, augmenting human labor to tackle workplace tasks with finesse — whether that’s automating routine stuff or helping humans crank out next-level decisions. Bottom line: AI’s the MVP in turbocharging knowledge work, unlocking new cash-flow avenues by turning smarts into stacks.\n\nSo, people’re not just using AI to grind harder — they’re hacking the future of work to *make money smarter*. That’s the rad revolution at play.'

Finally, we can evaluate the new chain on the same test set!

In [63]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'cooked-stove-87' at:
https://smith.langchain.com/o/d818ede9-bb51-42f5-821f-b15992a8e545/datasets/614eddd6-bbaf-45c6-9e36-1db976895173/compare?selectedSessions=ef5bb77d-09ca-4e5e-badb-7a540d00631d




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"Based on the rapid growth of ChatGPT, which re...","Yo, buckle up because the ChatGPT wave is stra...",None,"By July 2025, ChatGPT's usage had expanded sig...",1,1,1,6.848255,67e0c56d-6bb3-4b88-a4d6-e03c37afeca8,233b5ef8-a72f-4abf-b9aa-aecf26013506
1,"How do the 700 million users of ChatGPT, as of...","Alright, strap in for some next-level insight!...",None,"By July 2025, ChatGPT had been used weekly by ...",1,1,1,8.049222,f6aa2ef9-6ab9-4381-aa1f-1bd541b5e1b8,a0d10b79-a32d-411c-a100-9c69c0696fdd
2,"Based on the data, how has ChatGPT's usage in ...","Yo, buckle up — here’s the lowdown on ChatGPT’...",None,"Between 2024 and 2025, ChatGPT's usage in the ...",1,1,1,8.046252,f37baf0b-031a-4191-9efa-21182fd61d03,ef41aa0c-c3e0-47a5-bbf1-fcbf36e8814d
3,US ChatGPT use mostly for work or non-work and...,"Alright, buckle up for a wild ride through the...",None,"The context shows that by July 2025, about 70%...",0,0,1,7.884140,dabdd65f-00c1-4d6d-b1d4-375117c30290,e3187845-9796-4745-9650-9acd18bfe37f
4,How do occupation categories such as computer-...,"Alright, strap in for this turbo-charged break...",None,Variation in ChatGPT usage by occupation shows...,1,1,1,8.520567,c364651e-3bfd-48b2-9c74-16c42763fb79,3c4d888d-f098-4b79-ac6d-a9c81477b48b
5,Based on message volume comparison and message...,"Alright, let’s crank this up to eleven with so...",None,The data shows that non-work messages have gro...,1,1,1,6.600731,f3bacac5-c4de-4e9a-8fe1-81ccd9803485,d65f6c47-6c50-421b-a378-53270197a814
6,How do work-related message sharing patterns d...,"Alright, strap in because the breakdown here i...",None,Variation by occupation shows that users in hi...,1,1,1,8.802179,24ebe91b-be63-4845-8230-4bd53c637ad2,eca94d37-cb02-4f52-a89d-02b4abe56be2
7,How does the impact of occupation influence th...,"Alright, let’s dive deep into this data-driven...",None,The context indicates that ChatGPT usage varie...,1,1,1,8.064603,b8cb9ed1-c9fb-4fba-b2dd-839d298f8281,4d295262-8186-44a4-8f29-b7551b0f57b5
8,Whaat is the role of Computer Programming in t...,"Alright, buckle up because the role of Compute...",None,"According to the provided context, Computer Pr...",0,0,1,5.135800,5a40f1f4-2a26-4268-866e-650569a2286c,b8a7ade7-a072-4352-98bd-932efc665642
9,What is enginering and sciense in the context ...,"Yo, diving into the rad realm of ChatGPT usage...",None,Variation by Occupation Figure 23 presents var...,1,1,1,3.391691,7583638f-68f7-40d9-8074-e34f7958a9e5,1355cd84-6e32-4c9f-94ed-5623cd20c105


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

##### ✅ Answers:


Things that are now different
- Run 2: Each answer should be more dope because its in the prompt
- Run 2: uses a large embedding model, which allows better understanding of the test
- Run 2: chunk size is now larger, 1000 (vs 500)
- Run 2: The model is now stronger, 4.1 (vs 4o-mini), which should always produce equal or better outcomes

m

![title](image.png)
